# COMP47590: Advanced Machine Learning
# Assignment 1: Multi-label Classification

Name(s): Yili Lai, Qimeng Gan

Student Number(s): 15205910, 15205903

## Import Packages Etc

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math

from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.ensemble import BaggingClassifier
from sklearn.utils.validation import check_X_y, check_array, check_is_fitted
# import other useful packages

## Task 0: Load the Yeast Dataset

In [ ]:
dataset = pd.read_csv('yeast.csv')
data = np.array(dataset.iloc[:,:103])
functions = np.array(dataset.iloc[:,103:])
functions[18]

## Task 1: Implement the Binary Relevance Algorithm

In [ ]:
# Create a new classifier which is based on the sckit-learn BaseEstimator and ClassifierMixin classes
class BinaryRelevanceClassifier(BaseEstimator, ClassifierMixin):
    # Constructor for the classifier object
    def __init__(self, add_noise = False):
        self.add_noise = add_noise
        
    # The fit function to train a classifier
    def fit(self, data, functions):    
        # Create a new empty dictionary into which we will store relevance
        self.relevances_ = dict()

        # Iterate all functioins
        for i in range(14):
            status = functions[:,i]
            status_squeeze = np.squeeze(status)
            self.relevances_[i] = BaggingClassifier(n_estimators=10, random_state=0).fit(data, status_squeeze)
            
    # The predict function to make a set of predictions for a set of query instances
    def predict(self, X):
        # Check is fit had been called by confirming that the teamplates_ dictiponary has been set up
        check_is_fitted(self, ['relevances_'])

        # Initialise an empty list to store the predictions made
        pos_functions = list()
        
        # Iterate all functioins to predict
        for i in range(14):
            pos_functions.append(self.relevances_[i].predict(X))
            
        return np.array(pos_functions).T

In [ ]:
my_model = BinaryRelevanceClassifier()

In [ ]:
my_model.fit(np.array(dataset.iloc[:40,:103]), np.array(dataset.iloc[:40,103:]))

In [ ]:
my_model.predict(np.array(dataset.iloc[41:49,:103]))

## Task 2: Implement the Binary Relevance Algorithm with Under-Sampling

In [ ]:
# Write your code here


## Task 3: Compare the Performance of Different Binary Relevance Approaches

In [ ]:
# Write your code here


## Task 4: Implement the Classifier Chains Algorithm

In [ ]:
# Write your code here


## Task 5: Evaluate the Performance of the Classifier Chains Algorithm

In [ ]:
# Write your code here


## Task 6: Reflect on the Performance of the Different Models Evaluated

*Write your reflection here (max 300 words)*